In [1]:
# Importing the libraires
import numpy as np
import pandas as pd
import torch
import torch.nn  as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [7]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [8]:
nb_users

943

In [9]:
nb_movies

1682

In [10]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [11]:
training_set = convert(training_set)

In [12]:
training_set[:1]

[[0.0,
  3.0,
  4.0,
  3.0,
  3.0,
  0.0,
  4.0,
  1.0,
  5.0,
  0.0,
  2.0,
  0.0,
  5.0,
  0.0,
  5.0,
  5.0,
  0.0,
  4.0,
  5.0,
  0.0,
  1.0,
  4.0,
  0.0,
  0.0,
  4.0,
  3.0,
  0.0,
  4.0,
  1.0,
  3.0,
  0.0,
  5.0,
  0.0,
  2.0,
  1.0,
  0.0,
  2.0,
  3.0,
  0.0,
  3.0,
  2.0,
  5.0,
  4.0,
  0.0,
  5.0,
  4.0,
  0.0,
  5.0,
  0.0,
  5.0,
  0.0,
  4.0,
  0.0,
  0.0,
  5.0,
  0.0,
  5.0,
  4.0,
  5.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  4.0,
  0.0,
  4.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  4.0,
  0.0,
  4.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  5.0,
  4.0,
  5.0,
  0.0,
  0.0,
  0.0,
  5.0,
  2.0,
  4.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  2.0,
  4.0,
  0.0,
  0.0,
  5.0,
  1.0,
  5.0,
  0.0,
  0.0,
  0.0,
  5.0,
  3.0,
  0.0,
  0.0,
  5.0,
  0.0,
  0.0,
  3.0,
  4.0,
  5.0,
  0.0,
  2.0,
  5.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  4.0,
  0.0,
  4.0,
  3.0,
  5.0,
  1.0,
  3.0,
  0.0,
  3.0,
  2.0,
  0.0,

In [13]:
test_set = convert(test_set)

In [14]:
test_set[:1]

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  5.0,
  0.0,
  5.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  4.0,
  0.0,
  0.0,
  4.0,
  3.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  4.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  0.0,
  5.0,
  0.0,
  0.0,
  4.0,
  0.0,
  3.0,
  0.0,
  4.0,
  0.0,
  3.0,
  3.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  5.0,
  4.0,
  3.0,
  0.0,
  5.0,
  4.0,
  0.0,
  3.0,
  0.0,
  3.0,
  3.0,
  0.0,
  4.0,
  3.0,
  1.0,
  0.0,
  4.0,
  0.0,
  1.0,
  0.0,
  4.0,
  5.0,
  5.0,
  0.0,
  4.0,
  3.0,
  5.0,
  0.0,
  0.0,
  0.0,
  4.0,
  5.0,
  3.0,
  0.0,
  0.0,
  0.0,
  5.0,
  3.0,
  4.0,
  0.0,
  5.0,
  0.0,
  2.0,
  1.0,
  1.0,
  0.0,
  0.0,
  4.0,
  5.0,
  0.0,
  0.0,
  0.0,
  1.0,
  5.0,
  5.0,
  0.0,
  0.0,
  3.0,
  3.0,
  0.0,
  1.0,
  4.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  4.0,
  5.0,
  3.0,
  0.0,
  4.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,

In [15]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [16]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [17]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [18]:
# Creating the architecture of the Neural Network

In [19]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.full_connection_1 = nn.Linear(nb_movies, 20)
        self.full_connection_2 = nn.Linear(20, 10)
        self.full_connection_3 = nn.Linear(10, 20)
        self.full_connection_4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.full_connection_1(x))
        x = self.activation(self.full_connection_2(x))
        x = self.activation(self.full_connection_3(x))
        x = self.full_connection_4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [20]:
# Training the SAE (Stacked AutoEncoders)

In [21]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7714)
epoch: 2 loss: tensor(1.0966)
epoch: 3 loss: tensor(1.0535)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0309)
epoch: 6 loss: tensor(1.0265)
epoch: 7 loss: tensor(1.0238)
epoch: 8 loss: tensor(1.0217)
epoch: 9 loss: tensor(1.0206)
epoch: 10 loss: tensor(1.0197)
epoch: 11 loss: tensor(1.0188)
epoch: 12 loss: tensor(1.0183)
epoch: 13 loss: tensor(1.0177)
epoch: 14 loss: tensor(1.0175)
epoch: 15 loss: tensor(1.0173)
epoch: 16 loss: tensor(1.0168)
epoch: 17 loss: tensor(1.0167)
epoch: 18 loss: tensor(1.0164)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0160)
epoch: 21 loss: tensor(1.0161)
epoch: 22 loss: tensor(1.0160)
epoch: 23 loss: tensor(1.0161)
epoch: 24 loss: tensor(1.0158)
epoch: 25 loss: tensor(1.0157)
epoch: 26 loss: tensor(1.0154)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0148)
epoch: 29 loss: tensor(1.0136)
epoch: 30 loss: tensor(1.0120)
epoch: 31 loss: tensor(1.0098)
epoch: 32 loss: tensor(1.0098)
epoch: 33 loss: t

In [22]:
# Testing the SAE (Stacked AutoEncoders)

In [27]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9598)
